**N.B**
Due to the time limit on notebook, I choose 320,000 samples to save time.



In [ ]:
import pandas as pd
import numpy as np
import pickle
import gc #exposes the underlying memory management mechanism of Python, the automatic garbage collector. 
            #The module includes functions for controlling how the collector operates and to examine the objects known to the system, 
            #either pending collection or stuck in reference cycles and unable to be freed.
import re
from matplotlib import pyplot as plt
import xgboost as xgb
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import RMSprop

from sklearn.model_selection import train_test_split
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:

max_num_features = 10
pad_size = 1
boundary_letter = -1
space_letter = 0


df = pd.read_csv('/kaggle/input/en_train.csv.zip')

x_data = []
y_data =  pd.factorize(df['class'])#Encode the object as an enumerated type or categorical variable.
                                    #This method is useful for obtaining a numeric representation of an array when all that matters is identifying distinct values.

    
labels = y_data[1]
y_data = y_data[0]
gc.collect()#Although the garbage collector runs automatically as the interpreter executes your program, 
            #you may want to trigger collection to run at a specific time when you know there are a lot of objects to free 
            #or there is not much work happening in your application. Trigger collection using collect().
for x in df['before'].values:
    x_row = np.ones(max_num_features, dtype=int) * space_letter
    for xi, i in zip(list(str(x)), np.arange(max_num_features)):
        x_row[i] = ord(xi)
    x_data.append(x_row)


In [ ]:
len(x_data)

In [ ]:
max_data_size = 320000

In [ ]:

def context_window_transform(data, pad_size):
    pre = np.zeros(max_num_features)
    pre = [pre for x in np.arange(pad_size)]
    data = pre + data + pre
    neo_data = []
    for i in np.arange(len(data) - pad_size * 2):
        row = []
        for x in data[i : i + pad_size * 2 + 1]:
            row.append([boundary_letter])
            row.append(x)
        row.append([boundary_letter])
        neo_data.append([int(x) for y in row for x in y])
    return neo_data

x_data = x_data[:max_data_size]
y_data = y_data[:max_data_size]
x_data = np.array(context_window_transform(x_data, pad_size))
gc.collect()
x_data = np.array(x_data)
y_data = np.array(y_data)

print('Total number of samples:', len(x_data))
print('Use: ', max_data_size)
#x_data = np.array(x_data)
#y_data = np.array(y_data)

print('x_data sample:')
print(x_data[0])
print('y_data sample:')
print(y_data[0])
print('labels:')
print(labels)

In [ ]:
x_train = x_data
y_train = y_data
gc.collect()

x_train, x_valid, y_train, y_valid= train_test_split(x_train, y_train,
                                                      test_size=0.1, random_state=2017)
gc.collect()

In [ ]:

num_class = len(labels)
dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_valid, label=y_valid)
watchlist = [(dvalid, 'valid'), (dtrain, 'train')]

param = {'objective':'multi:softmax',
         'eta':'0.3', 'max_depth':10,
         'silent':1, 'nthread':-1,
         'num_class':num_class,
         'eval_metric':'merror'}
model = xgb.train(param, dtrain, 50, watchlist, early_stopping_rounds=20,
                  verbose_eval=10)


In [ ]:
gc.collect()

pred = model.predict(dvalid)
pred = [labels[int(x)] for x in pred]
y_valid = [labels[x] for x in y_valid]
x_valid = [ [ chr(x) for x in y[2 + max_num_features: 2 + max_num_features * 2]] for y in x_valid]
x_valid = [''.join(x) for x in x_valid]
x_valid = [re.sub('a+$', '', x) for x in x_valid]

gc.collect()

df_pred = pd.DataFrame(columns=['data', 'predict', 'target'])
df_pred['data'] = x_valid
df_pred['predict'] = pred
df_pred['target'] = y_valid


In [ ]:
df_pred.to_csv(os.path.join(r'.', 'pred.csv'))

df_erros = df_pred.loc[df_pred['predict'] != df_pred['target']]
df_erros.to_csv(os.path.join(r'.', 'errors.csv'), index=False)

model.save_model(os.path.join(r'.', 'xgb_model'))